In [1]:
import torch

In [11]:
def sample_and_weight():
    data = torch.tensor([1.1,2.1,2.0,1.9,0.0,-0.1, 0.05])
    likes = [None]*3
    mu_prior = torch.distributions.Normal(0.0, 10.0)
    sigma_prior = torch.distributions.gamma.Gamma(1.0, 1.0)
    for i in range(3):
        mu = mu_prior.sample()
        sigma = sigma_prior.sample()
        likes[i] = torch.distributions.Normal(mu, sigma)
    pi_prior = torch.distributions.dirichlet.Dirichlet(torch.tensor([1.0, 1.0, 1.0]))
    pi = pi_prior.sample()
    z_prior = torch.distributions.categorical.Categorical(pi)
    logW = 0
    zs = [None]*len(data)
    for i, y in enumerate(data):
        z = z_prior.sample()
        zs[i] = z
        likelihood = likes[z]
        logW += likelihood.log_prob(y)
    
    return zs[0] == zs[1], logW

In [12]:
sample_and_weight()

(tensor(False), tensor(-4980.4287))

In [15]:
results = torch.zeros(100000)
log_weights = torch.zeros(100000)

for i in range(100000):
    results[i], log_weights[i] = sample_and_weight()
    
print(results)

tensor([1., 1., 0.,  ..., 0., 1., 0.])


In [16]:
M = torch.max(log_weights)
nrm_wghts = torch.exp(log_weights-M)/torch.sum(torch.exp(log_weights-M))
expectation = torch.sum(nrm_wghts*results)
print(expectation)
print(torch.nonzero(nrm_wghts).size())

tensor(0.6238)
torch.Size([15687, 1])
